NN Feauture importance

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
 
path = "/content/gdrive/My Drive/Independent"
os.chdir(path)

Mounted at /content/gdrive


In [ ]:
%tensorflow_version 1.x

! pip install eli5

TensorFlow 1.x selected.
     |████████████████████████████████| 112kB 12.9MB/s 


In [4]:
import numpy as np
import tensorflow as tf
from scipy import sparse
from keras import backend as K
from keras.models import Sequential
from sklearn.utils import class_weight
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.wrappers.scikit_learn import KerasClassifier

def save(users_posts, users_label):
    sparse.save_npz("./result/v_countposts.npz", users_posts, False)

    file = open("./result/v_label.txt", "w")
    for user_label in users_label:
        file.write(str(user_label) + "\n")
    file.close()

def read_list(path):
    result = []
    with open(path, "r") as f:
        for line in f:
          temp = line.strip("\n")
          result.append(int(temp))
    return result

def read_npz(path):
    sparse_matrix = sparse.load_npz(path)
    return sparse_matrix

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

def create_model():
  model = Sequential()
  model.add(Dense(d, input_dim=d, activation='relu', kernel_initializer='random_normal'))
  model.add(Dropout(0.1))
  model.add(Dense(64, activation='relu', kernel_initializer='random_normal'))
  model.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
  model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
  model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
  return model

# load dataset
X= read_npz("./result/t_count_posts.npz")
y= read_list("./result/t_label.txt")

scaler = StandardScaler()
X = X.toarray()
X = X[:, 50000:]
X = scaler.fit_transform(X)
n, d= X.shape

model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=40, verbose=0)
model.fit(X,y)

perm = PermutationImportance(model, random_state=1).fit(X,y)
eli5.show_weights(perm)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Weight,Feature
0.0024 ± 0.0016,x580
0.0022 ± 0.0008,x711
0.0018 ± 0.0008,x212
0.0018 ± 0.0008,x662
0.0018 ± 0.0008,x731
0.0018 ± 0.0008,x338
0.0018 ± 0.0008,x794
0.0016 ± 0.0010,x596
0.0016 ± 0.0010,x296
0.0016 ± 0.0010,x927


In [7]:
import eli5
from eli5.sklearn import PermutationImportance

def create_model():
  model = Sequential()
  model.add(Dense(d, input_dim=d, activation='relu', kernel_initializer='random_normal'))
  model.add(Dropout(0.1))
  model.add(Dense(64, activation='relu', kernel_initializer='random_normal'))
  model.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
  model.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
  model.compile(loss='binary_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
  return model

# load dataset
X= read_npz("./result/t_tfidf_posts.npz")
y= read_list("./result/t_label.txt")

scaler = StandardScaler()
X = X.toarray()
X = X[:, 165000:170000]
X = scaler.fit_transform(X)
n, d= X.shape

model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=40, verbose=0)
model.fit(X,y)

perm = PermutationImportance(model, random_state=1).fit(X,y)
eli5.show_weights(perm)

Weight,Feature
0.0020 ± 0.0000,x2405
0.0018 ± 0.0023,x4345
0.0018 ± 0.0008,x4382
0.0016 ± 0.0010,x375
0.0014 ± 0.0020,x4392
0.0010 ± 0.0000,x2081
0.0010 ± 0.0000,x1595
0.0010 ± 0.0000,x2247
0.0010 ± 0.0000,x4411
0.0010 ± 0.0000,x2852
